In [1]:

from evaluators.library.graph_code_checker import GraphCodeCheckerEvaluator, InputSchema, OutputSchema
from AgentCodiaEvaluator.Agents.utils import llm_as_evaluator

parsed_notebooks = llm_as_evaluator.parse_notebooks([open("/Users/daniel/Desktop/Projects/xxxx/AgentCodiaEvaluator/demo_sample/fibonacci.ipynb", "r").read()])


06/26/2024 03:18:30 PM - DEBUG - This is a debug message. LOGGING INITIALIZED.
06/26/2024 03:18:30 PM - INFO - This is an info message. LOGGING INITIALIZED.


/Users/daniel/anaconda3/envs/evaluator/lib/python3.12/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_responses" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
a = GraphCodeCheckerEvaluator(
    name = "echo", # just any name
    config = {
        "graph_recursion": 50,
        "turn_workers": 4,
        "test_workers": 4
    },
    llm_config= {"provider":"openai_api", "model":"gpt-4o"},
    config_schema= {},
    input_schema = InputSchema.model_json_schema(),
    output_schema= OutputSchema.model_json_schema()
    
)

06/26/2024 03:18:35 PM - DEBUG - Initializing BaseEvaluator with name: echo
06/26/2024 03:18:35 PM - DEBUG - Validating schema
06/26/2024 03:18:35 PM - DEBUG - Schema: {}
06/26/2024 03:18:35 PM - DEBUG - Data: {'graph_recursion': 50, 'turn_workers': 4, 'test_workers': 4}


In [3]:
result = a.evaluate(
    input_data= {
        "metadata": parsed_notebooks[0]['metadata'],
        "conversation": parsed_notebooks[0]['conversation'],
    },
    config={ "happy_cases_count": 1, "edge_cases_count": 1,},
    format_to_issues_scores=True
    
)

06/26/2024 03:18:35 PM - INFO - Starting evaluation...
06/26/2024 03:18:35 PM - INFO - Validating input data...
06/26/2024 03:18:35 PM - DEBUG - Validating input data
06/26/2024 03:18:35 PM - DEBUG - Validating schema
06/26/2024 03:18:35 PM - DEBUG - Schema: {'description': 'This class represents the input schema for the graph code checker.\nIt includes metadata, conversation data, and counts for happy and edge cases.', 'properties': {'metadata': {'description': 'The metadata of the input.', 'title': 'Metadata'}, 'conversation': {'description': 'The conversation data.', 'items': {'type': 'object'}, 'title': 'Conversation', 'type': 'array'}, 'happy_cases_count': {'description': 'The number of happy cases', 'title': 'Happy Cases Count', 'type': 'integer'}, 'edge_cases_count': {'description': 'The number of edge cases', 'title': 'Edge Cases Count', 'type': 'integer'}}, 'required': ['metadata', 'conversation', 'happy_cases_count', 'edge_cases_count'], 'title': 'InputSchema', 'type': 'objec

###TURN:0->TEST0### I am calling the function `CodeExec`

###TURN:0->TEST0### {"CodeExec": {"run": {"exitcode": 0, "out": "Test case: n is 0\n", "error": "", "timing": 
{"real": 44, "user": 33, "sys": 14}}, "memory_usage": 11272192}}

06/26/2024 03:19:03 PM - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


###TURN:0->TEST0### {"test_code": "def print_fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        
print(a)\n        a, b = b, a + b\n\n# Test case: n is 0 (should handle gracefully without printing 
anything)\nprint(\"Test case: n is 0\")\nprint_fibonacci(0)", "result": "PASSED", "comment": "The function handled 
the case where n is 0 gracefully without printing anything."}

...

###TURN:0->TEST1### I am calling the function `CodeExec`

###TURN:0->TEST1### {"CodeExec": {"run": {"exitcode": 0, "out": "Output: 0\n\nExpected Output: 0\n\nTest Passed: 
True\n", "error": "", "timing": {"real": 42, "user": 27, "sys": 19}}, "memory_usage": 1757184}}

06/26/2024 03:19:15 PM - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


###TURN:0->TEST1### {"test_code": "import io\nimport sys\n\ndef print_fibonacci(n):\n    a, b = 0, 1\n    for _ in 
range(n):\n        print(a)\n        a, b = b, a + b\n\n# Test code\n# Redirect stdout to capture print 
statements\nold_stdout = sys.stdout\nnew_stdout = io.StringIO()\nsys.stdout = new_stdout\n\n# Call the function 
with n=1\nprint_fibonacci(1)\n\n# Reset stdout\nsys.stdout = old_stdout\n\n# Get the output and check if it matches
'0\\n'\noutput = new_stdout.getvalue()\nexpected_output = '0\\n'\n\nprint(f\"Output: {output}\")\nprint(f\"Expected
Output: {expected_output}\")\nprint(f\"Test Passed: {output == expected_output}\")", "result": "PASSED", "comment":
"The test passed successfully. The function print_fibonacci(1) correctly printed '0'."}

06/26/2024 03:19:23 PM - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Validating output schema.....
06/26/2024 03:19:23 PM - INFO - Formatting output...


In [4]:
for issue in result["issues"]:
    print("********************************")
    print(issue)


********************************
Turn: 0
What: The function print_fibonacci does not include a docstring.
Why: Docstrings are important for documenting the purpose and usage of the function, making the code more understandable.
Where: def print_fibonacci(n):
Severity: Low
Fix: Add a docstring to the function explaining its purpose and parameters.

********************************
Turn: 0
What: The function does not handle invalid input for n.
Why: If n is not a positive integer, the function may produce unexpected results or errors.
Where: def print_fibonacci(n):
Severity: Medium
Fix: Add input validation to ensure n is a positive integer.

********************************
Turn: 0
What: The function prints the Fibonacci sequence instead of returning it.
Why: Printing the sequence limits the usability of the function as it cannot be used programmatically.
Where: print(a)
Severity: Medium
Fix: Modify the function to return the Fibonacci sequence as a list instead of printing it.



In [5]:
print(result["score"])

3.0
